In [115]:
import skimage
import glob
import cv2
import os
import shutil
import skimage
import numpy as np
import pandas as pd

from skimage.io import imread, imsave

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
image_paths = glob.glob('../input/stage1_train/*/images/*.png')
train_shapes = []
for path in image_paths:
    img = cv2.imread(path)
#     img = skimage.io.imread(path)
    train_shapes.append(img.shape)
pd.Series(train_shapes).value_counts()

(256, 256, 3)      334
(256, 320, 3)      112
(520, 696, 3)       92
(360, 360, 3)       91
(1024, 1024, 3)     16
(512, 640, 3)       13
(603, 1272, 3)       6
(260, 347, 3)        5
(1040, 1388, 3)      1
dtype: int64

In [3]:
image_paths = glob.glob('../input/stage1_test/*/images/*.png')
test_shapes = []
for path in image_paths:
    img = cv2.imread(path)
#     img = skimage.io.imread(path)
    test_shapes.append(img.shape)
pd.Series(test_shapes).value_counts()

(256, 256, 3)    24
(512, 640, 3)     8
(512, 680, 3)     8
(260, 347, 3)     4
(520, 696, 3)     4
(520, 348, 3)     4
(519, 253, 3)     4
(524, 348, 3)     4
(519, 161, 3)     2
(519, 162, 3)     2
(390, 239, 3)     1
dtype: int64

### cropping

In [103]:
img_id = '9ab2d381f90b485a68b82bc07f94397a0373e3215ad20935a958738e55f3cfc2'
cv2.imread('../input/stage1_test/' + img_id + '/images/' + img_id + '.png').shape

(512, 640, 3)

In [134]:
exclude_ids = {'1cdbfee1951356e7b0a215073828695fe1ead5f8b1add119b6645d2fdc8d844e', 
               '505bc0a3928d8aef5ce441c5a611fdd32e1e8eccdc15cc3a52b88030acb50f81',
              '9ab2d381f90b485a68b82bc07f94397a0373e3215ad20935a958738e55f3cfc2'}
# exclude_ids = {}

In [135]:
test_crops_path = "../output/test_crops"
if not os.path.isdir(test_crops_path):
    os.mkdir(test_crops_path)

def image_split(path, exclude_ids):
    counter = 0
    img_id = path.split('/')[-1].split('.png')[0]
    img = cv2.imread(path)
    h, w = img.shape[0], img.shape[1]
    hn, wn = 0, 0
#     print(h, w)
    hi, wi = round(h / 256.0), round(w / 256.0)
    if (hi <= 1 and wi <= 1) or (img_id in exclude_ids):
        np.save('../output/test_crops/' + img_id + '.npy', img[:,:,:])
        counter += 1
#         print(img[:,:,:].shape)
    else:
        hn, wn = round(h*1.0 / (2*hi))*2, round(w*1.0 / (2*wi))*2
        for i in range(hi):
            for j in range(wi):
                crop = img[i*hn: min((i+1)*hn, h), j*wn: min((j+1)*wn, w), :]
#                 print(crop.shape)
                np.save('../output/test_crops/' + img_id + '_' + str(i) + '_' + str(j) + '.npy', crop)
                counter += 1
    return img_id, counter, h, w, hi, wi, hn, wn

In [138]:
image_paths = glob.glob('../input/stage1_test/*/images/*.png')
crop_dic = {}
crop_counter = 0
for path in image_paths:
    img_id, counter, h, w, hi, wi, hn, wn = image_split(path, exclude_ids)
    crop_counter += counter
    crop_dic[img_id] = (counter, h, w, hi, wi, hn, wn)
crop_counter

157

In [137]:
ori_test_img_ids = open('../input/test1_ids_gray2_53', 'r')
crop_test_img_ids = open('../output/test1_ids_gray2_53_crops', 'w')

img_id = 'id'
while img_id:
    img_id = ori_test_img_ids.readline()
    img_id = img_id.split('/')[-1].split('\n')[0]
#     print(img_id)
    if img_id in crop_dic:
        counter, h, w, hi, wi, hn, wn = crop_dic[img_id]
        if (hi <= 1 and wi <= 1) or (img_id in exclude_ids):
            crop_test_img_ids.write(img_id + '\n')
        else:
            for i in range(hi):
                for j in range(wi):
                    crop_test_img_ids.write(img_id + '_' + str(i) + '_' + str(j) + '\n')
crop_test_img_ids.close() 

### merge nps

In [151]:
merge_npy_path = "/home/jun/20180407/results/mask-rcnn-50-gray500-crop-01/submit/npys_merge"
if not os.path.isdir(merge_npy_path):
    os.mkdir(merge_npy_path)

In [152]:
gray_ids = []

ori_test_img_ids = open('../input/test1_ids_gray2_53', 'r')

img_id = 'id'
while img_id:
    img_id = ori_test_img_ids.readline()
    img_id = img_id.split('/')[-1].split('\n')[0]
    if img_id:
        gray_ids.append(img_id)

In [153]:
npy_path = '/home/jun/20180407/results/mask-rcnn-50-gray500-crop-01/submit/npys'
for key in gray_ids:
    counter, h, w, hi, wi, hn, wn = crop_dic[key]
    if counter == 1:
        shutil.copy(npy_path + os.sep + key + '.npy', merge_npy_path + os.sep + key + '.npy')
    else:
        mask = np.zeros((h,w))
        for i in range(hi):
            for j in range(wi):
                mask[i*hn: min((i+1)*hn, h), j*wn: min((j+1)*wn, w)] = np.load(npy_path + os.sep + key + \
                    '_' + str(i) + '_' + str(j) + '.npy')
        np.save(merge_npy_path + os.sep + key + '.npy', mask)
    

In [112]:
# crop_dic.keys()